### obesity dataset naive bayes clasifying

In [26]:
# import the nescessary libraries
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn import naive_bayes
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
#importing the dataset
file_path = 'ObesityDataSet_raw_and_data_sinthetic.csv'
data = pd.read_csv(file_path)
data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
print(data.dtypes)
print(data.shape)

Gender                             object
Age                               float64
Height                            float64
Weight                            float64
family_history_with_overweight     object
FAVC                               object
FCVC                              float64
NCP                               float64
CAEC                               object
SMOKE                              object
CH2O                              float64
SCC                                object
FAF                               float64
TUE                               float64
CALC                               object
MTRANS                             object
NObeyesdad                         object
dtype: object
(2111, 17)


In [4]:
# data processing and cleaning
data.isnull().sum()

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [5]:
transportation = data['MTRANS'].drop_duplicates().values.tolist()
transportation

['Public_Transportation', 'Walking', 'Automobile', 'Motorbike', 'Bike']

In [6]:
obeservations = data.shape[0]
mtrans_df = pd.DataFrame(columns=transportation)
for i in transportation:
    mtrans_df[i] = np.zeros(obeservations)
for i in transportation:
    for j in range(obeservations):
        column_number = mtrans_df.columns.get_loc(i)
        if data.iloc[j, -2] == i:
            mtrans_df.iloc[j, column_number] = 1
mtrans_df.head()            

,Public_Transportation,Walking,Automobile,Motorbike,Bike
0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0


In [7]:
new_data = pd.concat([data.iloc[:, :-2], mtrans_df, data.iloc[:, -1]], axis=1)
new_data.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,SCC,FAF,TUE,CALC,Public_Transportation,Walking,Automobile,Motorbike,Bike,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,...,no,0.0,1.0,no,1.0,0.0,0.0,0.0,0.0,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,...,yes,3.0,0.0,Sometimes,1.0,0.0,0.0,0.0,0.0,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,...,no,2.0,1.0,Frequently,1.0,0.0,0.0,0.0,0.0,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,...,no,2.0,0.0,Frequently,0.0,1.0,0.0,0.0,0.0,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,...,no,0.0,0.0,Sometimes,1.0,0.0,0.0,0.0,0.0,Overweight_Level_II


In [8]:
new_data['Gender'] = new_data['Gender'].replace('Female', 1)
new_data['Gender'] = new_data['Gender'].replace('Male', 0)
new_data['Gender'] = new_data['Gender'].astype('int')
new_data['Gender'].head()

0    1
1    1
2    0
3    0
4    0
Name: Gender, dtype: int32

In [9]:
new_data = new_data.drop('Height', 1)
new_data = new_data.drop('Weight', 1)
new_data.head()

,Gender,Age,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,Public_Transportation,Walking,Automobile,Motorbike,Bike,NObeyesdad
0,1,21.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,1.0,0.0,0.0,0.0,0.0,Normal_Weight
1,1,21.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,1.0,0.0,0.0,0.0,0.0,Normal_Weight
2,0,23.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,1.0,0.0,0.0,0.0,0.0,Normal_Weight
3,0,27.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,0.0,1.0,0.0,0.0,0.0,Overweight_Level_I
4,0,22.0,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,1.0,0.0,0.0,0.0,0.0,Overweight_Level_II


In [10]:
new_data['family_history_with_overweight'] = new_data['family_history_with_overweight'].replace('yes', 1)
new_data['family_history_with_overweight'] = new_data['family_history_with_overweight'].replace('no', 0)
new_data['family_history_with_overweight'] = new_data['family_history_with_overweight'].astype('int')
new_data['family_history_with_overweight'].head()

0    1
1    1
2    1
3    0
4    0
Name: family_history_with_overweight, dtype: int32

In [11]:
new_data['FAVC'] = new_data['FAVC'].replace('yes', 1)
new_data['FAVC'] = new_data['FAVC'].replace('no', 0)
new_data['FAVC'] = new_data['FAVC'].astype('int')
new_data['FAVC'].head()

0    0
1    0
2    0
3    0
4    0
Name: FAVC, dtype: int32

In [12]:
frequency = ['no', 'Sometimes', 'Frequently', 'Always']
print(frequency)

['no', 'Sometimes', 'Frequently', 'Always']


In [13]:
for i in range(len(frequency)):
    new_data['CAEC'] = new_data['CAEC'].replace(frequency[i], i)
new_data['CAEC'] = new_data['CAEC'].astype('int')
new_data['CAEC'].head()

0    1
1    1
2    1
3    1
4    1
Name: CAEC, dtype: int32

In [14]:
new_data['SMOKE'] = new_data['SMOKE'].replace('yes', 1)
new_data['SMOKE'] = new_data['SMOKE'].replace('no', 0)
new_data['SMOKE'] = new_data['SMOKE'].astype('int')
new_data['SMOKE'].head()

0    0
1    1
2    0
3    0
4    0
Name: SMOKE, dtype: int32

In [15]:
new_data['SCC'] = new_data['SCC'].replace('yes', 1)
new_data['SCC'] = new_data['SCC'].replace('no', 0)
new_data['SCC'] = new_data['SCC'].astype('int')
new_data['SCC'].head()

0    0
1    1
2    0
3    0
4    0
Name: SCC, dtype: int32

In [16]:
for i in range(len(frequency)):
    new_data['CALC'] = new_data['CALC'].replace(frequency[i], i)
new_data['CALC'] = new_data['CALC'].astype('int')
new_data['CALC'].head()

0    0
1    1
2    2
3    2
4    1
Name: CALC, dtype: int32

In [17]:
new_data.head()

,Gender,Age,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,Public_Transportation,Walking,Automobile,Motorbike,Bike,NObeyesdad
0,1,21.0,1,0,2.0,3.0,1,0,2.0,0,0.0,1.0,0,1.0,0.0,0.0,0.0,0.0,Normal_Weight
1,1,21.0,1,0,3.0,3.0,1,1,3.0,1,3.0,0.0,1,1.0,0.0,0.0,0.0,0.0,Normal_Weight
2,0,23.0,1,0,2.0,3.0,1,0,2.0,0,2.0,1.0,2,1.0,0.0,0.0,0.0,0.0,Normal_Weight
3,0,27.0,0,0,3.0,3.0,1,0,2.0,0,2.0,0.0,2,0.0,1.0,0.0,0.0,0.0,Overweight_Level_I
4,0,22.0,0,0,2.0,1.0,1,0,2.0,0,0.0,0.0,1,1.0,0.0,0.0,0.0,0.0,Overweight_Level_II


In [18]:
x = new_data.iloc[:,:-1]
y = new_data.iloc[:,-1]

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [30]:
classifier = naive_bayes.GaussianNB()
classifier.fit(x_train,y_train)
%timeit classifier.fit(x_train,y_train)
predicted_y = classifier.predict(x_test)
classifier.score(x_test, y_test)



8.34 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


0.4373522458628842

In [29]:
scores = cross_val_score(classifier, x_train, y_train, cv=5)
scores

array([0.38461538, 0.40828402, 0.39940828, 0.37091988, 0.43323442])